# Extracting news articles' urls using MediaStack API

In [29]:
# Python 3
import http.client, urllib.parse

conn = http.client.HTTPConnection('api.mediastack.com')

params = urllib.parse.urlencode({
    'access_key': '21ce0b29c198c51e7ccec2a30c617c84',
    'categories': 'business',
    'sort': 'published_desc',
    'languages': 'en',
    # 'keywords': 'finance',
    # "date": "2024-10-05,2024-11-05",
    'sources': '-seekingalpha', 
    'limit': 100,
    })

conn.request('GET', '/v1/news?{}'.format(params))

res = conn.getresponse()
data1 = res.read()

print(data1.decode('utf-8'))

{"pagination":{"limit":100,"offset":0,"count":100,"total":10000},"data":[{"author":"PTI","title":"Market Infrastructure Institutions Delay Implementation Of Direct Securities Payouts","description":"Market Infrastructure Institutions Delay Implementation Of Direct Securities Payouts","url":"https:\/\/www.ndtvprofit.com\/markets\/market-infrastructure-institutions-delay-implementation-of-direct-securities-payouts","source":"Bloomberg | Latest And Live Business","image":null,"category":"business","language":"en","country":"us","published_at":"2024-11-12T15:23:41+00:00"},{"author":"PTI","title":"India Successfully Test-Fires Long-Range Cruise Missile Off Odisha Coast","description":"India Successfully Test-Fires Long-Range Cruise Missile Off Odisha Coast","url":"https:\/\/www.ndtvprofit.com\/nation\/india-successfully-test-fires-long-range-cruise-missile-off-odisha-coast","source":"Bloomberg | Latest And Live Business","image":null,"category":"business","language":"en","country":"us","pub

# Scraping text content from the news url

In [ ]:
import http.client
import urllib.parse
import json
import requests
from bs4 import BeautifulSoup


# Step 2: Decode and load the data into a dictionary
decoded_data = data1.decode('utf-8')
news_data = json.loads(decoded_data)

# Step 3: Define a function to scrape article body from the URL
def fetch_article_body(url):
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for a successful request
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # This is a generic way to capture article paragraphs; modify as needed
        paragraphs = soup.find_all('p')
        article_body = ' '.join(paragraph.text for paragraph in paragraphs)
        
        
        return article_body.strip()
    
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

i=1

# Step 4: Iterate over each entry in 'data' and add the 'body' field
for item in news_data['data']:
    print(i)
    i+=1
    item['body'] = fetch_article_body(item['url'])

# Step 5: Save the updated data with article bodies to a JSON file
with open('first_corpus.json', 'w') as file:
    json.dump(news_data, file, indent=4)

# Constructing claims

In [ ]:
import json

def convert_to_json(content):
    # Replace newline characters with spaces
    content = content.replace('\n', '')

    # Parse the string into a dictionary
    try:
        # Use json.loads() to parse the JSON string
        json_content = json.loads(content)
        return json_content
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return None

def get_claims(item):
    client = OpenAI()

    completion = client.chat.completions.create(
        model="gpt-4o",
        # api_key = os.environ.get("OPENAI_API_KEY"),
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "max_completion_tokens": 1024,
                "content": 
                f"""
                {item}

                A "claim" is a statement or assertion made within a text expressing a belief, opinion, or fact. Given
    the news article above, please extract as many claims as you can (make more than 1 claims) and their associated topics.
    Note: The claim should not contain ambiguous references, such as ’he’,’ she,’ and’ it’, and should use
    complete names. If there are multiple topics, give the most dominant one. The target of the claim (one
    entity) is the specific individual, group, or organization that the statement or assertion within a text is
    directed towards or about which it is making a case. The topic of the claim should be a simple phrase
    representing the claim’s central argument concept. Please
    generate a claim based on the given evidence. Don’t generate the evidence yourself.
    Please give the response following json format:

    claims:[
        [
        author: [you can get this from the given news article]
        title: [you can get this from the given news article]
        description:[you can get this from the given news article]
        url: [you can get this from the given news article]
        source: [you can get this from the given news article]
        image: [you can get this from the given news article]
        category: [you can get this from the given news article]
        language: [you can get this from the given news article]
        country: [you can get this from the given news article]
        published_at: [you can get this from the given news article]
        Evidence: [the relevant sentence, dont make this very long]
        Claims: [extract claim]
        Claim Target: [target]
        Claim Topic: [topic]
        ]

        [
        author: [you can get this from the given news article]
        title: [you can get this from the given news article]
        description:[you can get this from the given news article]
        url: [you can get this from the given news article]
        source: [you can get this from the given news article]
        image: [you can get this from the given news article]
        category: [you can get this from the given news article]
        language: [you can get this from the given news article]
        country: [you can get this from the given news article]
        published_at: [you can get this from the given news article]
        Evidence: [the relevant sentence, dont make this very long]
        Claims: [second claim]
        Claim Target: [target]
        Claim Topic: [topic]
        ]
    ]

    and this keeps on repeating, so that your output is a json object having multiple claim dictionaries inside it

                """
            }
        ],

        response_format={ "type": "json_object" }
    )

    # print(completion.choices[0].message)
    return completion.choices[0].message.content

# Initialize an empty list to store claims
claims = []
i=1

for item in news_data['data']:
    ans = get_claims(item)
    # print(ans)
    json_ans = convert_to_json(ans)
    print(json_ans)
    print("\n\n\n")
    print(i)
    i+=1
    if json_ans is not None:
        claims.append(json_ans)

# # claims now contains a list of all processed claim dictionaries

with open('claims.json', 'w') as f:
    json.dump(claims, f, indent=4)

print("Claims saved")


In [4]:
import json

with open('claims.json', 'r') as file:
    claims = json.load(file)


In [5]:
claims

[{'claims': [{'author': 'Claire James',
    'title': 'AI Revolution in Healthcare, Finance, and Law: How Duality Series is Leading the Charge',
    'description': 'In an era where artificial intelligence is reshaping entire industries, The Duality Companies, Inc. is at the forefront with its newly expanded Duality Series. This suite of AI-powered tools—Duality Health, Duality Finance, and Duality Law—aims to bring data-driven precision and efficiency to healthcare, finance, and legal sectors, redefining traditional workflows and decision-making processes. The AI-Powered [&#8230;]The post AI Revolution in Healthcare, Finance, and Law: How Duality Series is Leading the Charge appeared first on ABC Money.',
    'url': 'https://www.abcmoney.co.uk/2024/11/ai-revolution-in-healthcare-finance-and-law-how-duality-series-is-leading-the-charge/',
    'source': 'ABC Money',
    'image': None,
    'category': 'business',
    'language': 'en',
    'country': 'uk',
    'published_at': '2024-11-11T14

In [6]:
filtered_claims = []

for claim_set in claims:
    for claim in claim_set['claims']:
        filtered_claim = {key: claim[key] for key in ['url', 'published_at','Evidence', 'Claims', 'Claim Target', 'Claim Topic'] if key in claim}
        filtered_claims.append(filtered_claim)         

In [4]:
filtered_claims

[{'url': 'https://www.abcmoney.co.uk/2024/11/ai-revolution-in-healthcare-finance-and-law-how-duality-series-is-leading-the-charge/',
  'published_at': '2024-11-11T14:56:47+00:00',
  'Evidence': 'The Duality Companies, Inc. is at the forefront with its newly expanded Duality Series.',
  'Claims': 'The Duality Companies, Inc. is leading the charge in revolutionizing industries with its Duality Series.',
  'Claim Target': 'The Duality Companies, Inc.',
  'Claim Topic': 'Leadership in AI innovation'},
 {'url': 'https://www.abcmoney.co.uk/2024/11/ai-revolution-in-healthcare-finance-and-law-how-duality-series-is-leading-the-charge/',
  'published_at': '2024-11-11T14:56:47+00:00',
  'Evidence': 'Duality Health has been operational at Clearway Pain Solutions.',
  'Claims': 'Duality Health is enhancing efficiency in healthcare by being operational at Clearway Pain Solutions.',
  'Claim Target': 'Duality Health',
  'Claim Topic': 'AI in healthcare'},
 {'url': 'https://www.abcmoney.co.uk/2024/11/

# Making claim sets

In [7]:
import json
from collections import defaultdict



# Group claims by Claim Target and Claim Topic
grouped_claims = defaultdict(list)

for claim in filtered_claims:
    grouped_claims[claim['Claim Target']].append(claim)
    grouped_claims[claim['Claim Topic']].append(claim)

# Generate claim_sets structure with unique URLs within each group
claim_sets = []
for common, claims in grouped_claims.items():
    unique_urls = {}
    for claim in claims:
        if claim["url"] not in unique_urls:
            unique_urls[claim["url"]] = claim

    # Only include groups with at least 2 unique URLs
    if len(unique_urls) > 1:
        claim_set = {
            "common": common,
            "list_of_claims": [{"published_at": claim["published_at"],"url": claim["url"], "Claims": claim["Claims"]} for claim in unique_urls.values()]
        }
        claim_sets.append(claim_set)

# Final output
output = {"claim_sets": claim_sets}

# Convert to JSON format for display
# print(json.dumps(output, indent=4))
output

{'claim_sets': [{'common': 'Stock Performance',
   'list_of_claims': [{'published_at': '2024-11-11T14:47:56+00:00',
     'url': 'https://www.investing.com/news/company-news/ngne-stock-soars-to-52week-high-reaching-7372-amidst-surge-93CH-3714581',
     'Claims': 'NGNE stock has reached a 52-week high of $73.72.'},
    {'published_at': '2024-11-11T14:46:29+00:00',
     'url': 'https://www.investing.com/news/company-news/prodex-stock-soars-to-alltime-high-of-468-amidst-robust-growth-93CH-3714579',
     'Claims': 'Pro-Dex shares have reached an all-time high of $46.8.'},
    {'published_at': '2024-11-11T14:45:56+00:00',
     'url': 'https://www.investing.com/news/company-news/aroc-stock-hits-52week-high-at-2349-amid-robust-growth-93CH-3714574',
     'Claims': 'Archrock Inc. stock has reached a 52-week high.'},
    {'published_at': '2024-11-11T14:45:53+00:00',
     'url': 'https://www.investing.com/news/company-news/blackrock-floating-rate-stock-hits-52week-high-at-1417-93CH-3714571',
     

# Inference query

In [15]:
from openai import OpenAI

def convert_to_json(content):
    # Replace newline characters with spaces
    content = content.strip()
    content = content.replace('\n', '')
    content = content.strip()
    print(f"Content to parse: {content}")  # Debugging print

    # Parse the string into a dictionary
    try:
        # Use json.loads() to parse the JSON string
        json_content = json.loads(content)
        return json_content
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return None
    

    # content = content.replace('\n', '')  # or use content.replace('\n', ' ') to preserve spaces
    # print(f"Content to parse: {content}")  # Debugging print

    # try:
    #     json_content = json.loads(content)
    #     return json_content
    # except json.JSONDecodeError as e:
    #     print(f"Error decoding JSON: {e}")
    #     print(f"Invalid JSON: {content}")  # Inspect the invalid content
    #     return None

def make_cqueries(item):
    client = OpenAI()

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        # api_key = os.environ.get("OPENAI_API_KEY"),
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "max_completion_tokens": 2048,
                "content": 
                f"""

                {item}
                
                
                A multi-hop question is a query requiring multiple inferential leaps or accessing several pieces of
    information from different locations or sources to arrive at an answer. The following are news articles’
    metadata and claims come from the articles. All the claims from the article are related to a similar
    target. 
    
  Your task is to generate one multi-hop inference question based on the claims. Here are some
instructions:
1. Find the Connection: The connection between claims is <common>, which is how these key pieces of
information are related or how they can be combined to form a more complex idea.
2. Formulate the Question: Create a question that cannot be answered by relying on just one of the
sentences but instead requires understanding and linking the information from all of the sources. The
answer is <common>.
3. Ensure Coherence: Make sure the question flows logically from the combined information and is
clear and unambiguous.
    
    To generate a inference
question from claims, you need to use the following keywords: "claim", "published_at"


    output just the answer in json form (dictionary), use curly brackets, dont give extra spaces betweeen the words:

[
  [
        
    query: ["the question"],
    answer: ["the answer, yes or no"],
    question_type: ["temporal_query"],
      
    evidence_list:
    [
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
      ],

    
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
          
      ]
    ]
  ],
            
  [
    query: ["the question"],
    answer: ["the answer, yes or no"],
    question_type: ["comparison_query"],
    evidence_list:
    [
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
      ],
      
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
      ]
    ]
  ]
    
]

    in a similar way you can output more queries in json format

    output just the answer, your answer must contain the json object only, dont begin with "here is the answer"



                """
            }
        ],

        response_format={ "type": "json_object" }
    )

    # print(completion.choices[0].message)
    return completion.choices[0].message.content



In [16]:
queries = []

for item in output['claim_sets']:
    iq= make_cqueries(item)
    json_iq = convert_to_json(iq)

    if json_iq is not None:
        queries.append(json_iq)
        
    

Content to parse: {  "query": "What is the status of the stock performance for the mentioned companies?",  "answer": "Stock Performance",  "question_type": "temporal_query",  "evidence_list": [    {      "url": "https://www.investing.com/news/company-news/ngne-stock-soars-to-52week-high-reaching-7372-amidst-surge-93CH-3714581",      "published_at": "2024-11-11T14:47:56+00:00",      "claim": "NGNE stock has reached a 52-week high of $73.72."    },    {      "url": "https://www.investing.com/news/company-news/prodex-stock-soars-to-alltime-high-of-468-amidst-robust-growth-93CH-3714579",      "published_at": "2024-11-11T14:46:29+00:00",      "claim": "Pro-Dex shares have reached an all-time high of $46.8."    },    {      "url": "https://www.investing.com/news/company-news/aroc-stock-hits-52week-high-at-2349-amid-robust-growth-93CH-3714574",      "published_at": "2024-11-11T14:45:56+00:00",      "claim": "Archrock Inc. stock has reached a 52-week high."    },    {      "url": "https://www.

# Comparison query

In [2]:
from openai import OpenAI

def convert_to_json(content):
    # Replace newline characters with spaces
    content = content.strip()
    content = content.replace('\n', '')
    content = content.strip()
    print(f"Content to parse: {content}")  # Debugging print

    # Parse the string into a dictionary
    try:
        # Use json.loads() to parse the JSON string
        json_content = json.loads(content)
        return json_content
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return None
    

    # content = content.replace('\n', '')  # or use content.replace('\n', ' ') to preserve spaces
    # print(f"Content to parse: {content}")  # Debugging print

    # try:
    #     json_content = json.loads(content)
    #     return json_content
    # except json.JSONDecodeError as e:
    #     print(f"Error decoding JSON: {e}")
    #     print(f"Invalid JSON: {content}")  # Inspect the invalid content
    #     return None

def make_cqueries(item):
    client = OpenAI()

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        # api_key = os.environ.get("OPENAI_API_KEY"),
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "max_completion_tokens": 2048,
                "content": 
                f"""

                {item}
                
                
                A multi-hop question is a query requiring multiple inferential leaps or accessing several pieces of
    information from different locations or sources to arrive at an answer. The following are news articles’
    metadata and claims come from the articles. All the claims from the article are related to a similar
    target. Your task is to generate as many comparison questions as you can based on all the
    claims from different sources. This question needs to compare some factual elements of the claims that
    are explicitly stated to find where they agree or differ. The correct answer to this question is expressed
    as a comparative adjective, a statement of alignment, a simple yes or no. To generate a comparative
question from claims, you need to use the following keywords: "claim", "published_at"


    output just the answer in json form (dictionary), use curly brackets, dont give extra spaces betweeen the words:

[
  [
        
    query: ["the question"],
    answer: ["the answer, yes or no"],
    question_type: ["comparison_query"],
      
    evidence_list:
    [
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
      ],

    
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
          
      ]
    ]
  ],
            
  [
    query: ["the question"],
    answer: ["the answer, yes or no"],
    question_type: ["comparison_query"],
    evidence_list:
    [
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
      ],
      
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
      ]
    ]
  ]
    
]

    in a similar way you can output more queries in json format

    output just the answer, your answer must contain the json object only, dont begin with "here is the answer"



                """
            }
        ],

        response_format={ "type": "json_object" }
    )

    # print(completion.choices[0].message)
    return completion.choices[0].message.content



In [8]:
queries = []

for item in output['claim_sets']:
    cq= make_cqueries(item)
    json_cq = convert_to_json(cq)

    if json_cq is not None:
        queries.append(json_cq)
        
    

Content to parse: {    "query": ["Is the claim that NGNE stock has reached a 52-week high of $73.72 the same as the claim that Pro-Dex shares have reached an all-time high of $46.8?"],    "answer": ["no"],    "question_type": ["comparison_query"],    "evidence_list": [        {            "url": "https://www.investing.com/news/company-news/ngne-stock-soars-to-52week-high-reaching-7372-amidst-surge-93CH-3714581",            "published_at": "2024-11-11T14:47:56+00:00",            "claim": "NGNE stock has reached a 52-week high of $73.72."        },        {            "url": "https://www.investing.com/news/company-news/prodex-stock-soars-to-alltime-high-of-468-amidst-robust-growth-93CH-3714579",            "published_at": "2024-11-11T14:46:29+00:00",            "claim": "Pro-Dex shares have reached an all-time high of $46.8."        }    ]}
Content to parse: {  "query": ["Do investors support Pro-Dex due to strong financial results and strategic initiatives as claimed on 2024-11-11?"],  

In [10]:
queries

[{'query': ['Is the claim that NGNE stock has reached a 52-week high of $73.72 the same as the claim that Pro-Dex shares have reached an all-time high of $46.8?'],
  'answer': ['no'],
  'question_type': ['comparison_query'],
  'evidence_list': [{'url': 'https://www.investing.com/news/company-news/ngne-stock-soars-to-52week-high-reaching-7372-amidst-surge-93CH-3714581',
    'published_at': '2024-11-11T14:47:56+00:00',
    'claim': 'NGNE stock has reached a 52-week high of $73.72.'},
   {'url': 'https://www.investing.com/news/company-news/prodex-stock-soars-to-alltime-high-of-468-amidst-robust-growth-93CH-3714579',
    'published_at': '2024-11-11T14:46:29+00:00',
    'claim': 'Pro-Dex shares have reached an all-time high of $46.8.'}]},
 {'query': ['Do investors support Pro-Dex due to strong financial results and strategic initiatives as claimed on 2024-11-11?'],
  'answer': ['yes'],
  'question_type': ['comparison_query'],
  'evidence_list': [{'url': ['https://www.investing.com/news/comp

# Temporal query

In [12]:
from openai import OpenAI

def convert_to_json(content):
    # Replace newline characters with spaces
    content = content.strip()
    content = content.replace('\n', '')
    content = content.strip()
    print(f"Content to parse: {content}")  # Debugging print

    # Parse the string into a dictionary
    try:
        # Use json.loads() to parse the JSON string
        json_content = json.loads(content)
        return json_content
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return None
    

    # content = content.replace('\n', '')  # or use content.replace('\n', ' ') to preserve spaces
    # print(f"Content to parse: {content}")  # Debugging print

    # try:
    #     json_content = json.loads(content)
    #     return json_content
    # except json.JSONDecodeError as e:
    #     print(f"Error decoding JSON: {e}")
    #     print(f"Invalid JSON: {content}")  # Inspect the invalid content
    #     return None

def make_cqueries(item):
    client = OpenAI()

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        # api_key = os.environ.get("OPENAI_API_KEY"),
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "max_completion_tokens": 2048,
                "content": 
                f"""

                {item}
                
                
                A multi-hop question is a query requiring multiple inferential leaps or accessing several pieces of
    information from different locations or sources to arrive at an answer. The following are news articles’
    metadata and claims come from the articles. All the claims from the article are related to a similar
    target. 
    
  Please create a time-sensitive comparison questions using data provided. That is to compare the consistency or sequence of reports on similar topics at multiple different time points. If it is to compare the consistency, please clearly mention the news source and time in the
  question. If it is to compare sequences of reports, just clearly mention the news
  source and do not mention the timeline. The correct answer should based on the factual excerpts and is only one word.
    
    To generate a temporal
question from claims, you need to use the following keywords: "claim", "published_at"


    output just the answer in json form (dictionary), use curly brackets, dont give extra spaces betweeen the words:

[
  [
        
    query: ["the question"],
    answer: ["the answer, yes or no"],
    question_type: ["temporal_query"],
      
    evidence_list:
    [
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
      ],

    
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
          
      ]
    ]
  ],
            
  [
    query: ["the question"],
    answer: ["the answer, yes or no"],
    question_type: ["comparison_query"],
    evidence_list:
    [
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
      ],
      
      [
        url: ["you will get this from the item"],
        published_at: ["you will get this from the item"],
        claim: ["you will get this from the item"]
      ]
    ]
  ]
    
]

    in a similar way you can output more queries in json format

    output just the answer, your answer must contain the json object only, dont begin with "here is the answer"



                """
            }
        ],

        response_format={ "type": "json_object" }
    )

    # print(completion.choices[0].message)
    return completion.choices[0].message.content



In [13]:
queries = []

for item in output['claim_sets']:
    tq= make_cqueries(item)
    json_tq = convert_to_json(tq)

    if json_tq is not None:
        queries.append(json_tq)
        
    

Content to parse: {  "query": "Are the claims consistent in reporting the stock reaching highs across different companies?",  "answer": "yes",  "question_type": "comparison_query",  "evidence_list": [    {      "url": "https://www.investing.com/news/company-news/ngne-stock-soars-to-52week-high-reaching-7372-amidst-surge-93CH-3714581",      "published_at": "2024-11-11T14:47:56+00:00",      "claim": "NGNE stock has reached a 52-week high of $73.72."    },    {      "url": "https://www.investing.com/news/company-news/prodex-stock-soars-to-alltime-high-of-468-amidst-robust-growth-93CH-3714579",      "published_at": "2024-11-11T14:46:29+00:00",      "claim": "Pro-Dex shares have reached an all-time high of $46.8."    }  ]}
Content to parse: {  "query": "Are the reports consistent with each other?",  "answer": "Yes",  "question_type": "temporal_query",  "evidence_list": [    {      "url": "https://www.investing.com/news/company-news/prodex-stock-soars-to-alltime-high-of-468-amidst-robust-gro

In [14]:
queries

[{'query': 'Are the claims consistent in reporting the stock reaching highs across different companies?',
  'answer': 'yes',
  'question_type': 'comparison_query',
  'evidence_list': [{'url': 'https://www.investing.com/news/company-news/ngne-stock-soars-to-52week-high-reaching-7372-amidst-surge-93CH-3714581',
    'published_at': '2024-11-11T14:47:56+00:00',
    'claim': 'NGNE stock has reached a 52-week high of $73.72.'},
   {'url': 'https://www.investing.com/news/company-news/prodex-stock-soars-to-alltime-high-of-468-amidst-robust-growth-93CH-3714579',
    'published_at': '2024-11-11T14:46:29+00:00',
    'claim': 'Pro-Dex shares have reached an all-time high of $46.8.'}]},
 {'query': 'Are the reports consistent with each other?',
  'answer': 'Yes',
  'question_type': 'temporal_query',
  'evidence_list': [{'url': 'https://www.investing.com/news/company-news/prodex-stock-soars-to-alltime-high-of-468-amidst-robust-growth-93CH-3714579',
    'published_at': '2024-11-11T14:46:29+00:00',
   

In [17]:
filename = 'dataset.json'

# Save the `queries` list to a JSON file
with open(filename, 'w') as json_file:
    json.dump(queries, json_file, indent=4)

print(f"Queries have been saved to {filename}")

Queries have been saved to dataset.json


In [18]:
queries

[{'query': 'What is the status of the stock performance for the mentioned companies?',
  'answer': 'Stock Performance',
  'question_type': 'temporal_query',
  'evidence_list': [{'url': 'https://www.investing.com/news/company-news/ngne-stock-soars-to-52week-high-reaching-7372-amidst-surge-93CH-3714581',
    'published_at': '2024-11-11T14:47:56+00:00',
    'claim': 'NGNE stock has reached a 52-week high of $73.72.'},
   {'url': 'https://www.investing.com/news/company-news/prodex-stock-soars-to-alltime-high-of-468-amidst-robust-growth-93CH-3714579',
    'published_at': '2024-11-11T14:46:29+00:00',
    'claim': 'Pro-Dex shares have reached an all-time high of $46.8.'},
   {'url': 'https://www.investing.com/news/company-news/aroc-stock-hits-52week-high-at-2349-amid-robust-growth-93CH-3714574',
    'published_at': '2024-11-11T14:45:56+00:00',
    'claim': 'Archrock Inc. stock has reached a 52-week high.'},
   {'url': 'https://www.investing.com/news/company-news/blackrock-floating-rate-stock-